# SharePoint to WordPress: Datasets

This notebook generates an XML ready to be imported into WordPress, Datasets as products on Woocommerce, with all the Taxonomy of the Attributes encoded as WordPress import requires.

In [41]:
import pandas as pd
import json

In [42]:
filename = "SP_DatasetRegistry.csv"
df = pd.read_csv(filename, low_memory=False)

In [43]:
#BL_SP_Fields = 'UniqueID,BL Dataset Provider,DAR,Dataset Title,User Access Level,BL Labs Assistance ,Type Of Dataset,Year Span,Authors,Contributors,Short Description,Creator,Additional Information,Language,Date Added,Last Time Updated,Created By,Contact Person,Department,Ownership,Location,Official URL,Related URLs,Location Notes,Is It Being Updated?,Any Issues With Access?,Image Res,Data Improvement,Data Enhancement,Format,Files,Files Size,Divided Into Smaller Bits?,Quality,T&C Needed?,Rights Status,Rights Assessment,Commercial?,Data Sensitivity,Data Protection,Origins/Selection of Dataset,Omitted Content and Reasons,Represented Groups,Image For Web,Keywords,Published on Wiki?,Published on Flickr?'
#WP_Import_Fields = 'UniqueID,pa_bldatasetprovider,DAR,Dataset Title,pa_useraccesslevel,pa_bllabsassistance,pa_typeofdataset,pa_yearspan,Authors,Contributors,Short Description,Institution,Additional Information,pa_language,Date Added,Last Time Updated,Created By,Contact Person,Department,Ownership,Location,Official URL,Related URLs,Location Notes,pa_isitbeingupdated,Any Issues With Access?,pa_imageres,Data Improvement,Data Enhancement,pa_format,Files,Files Size,Divided Into Smaller Bits?,Quality,pa_tcneeded,pa_rightsdstatus,parightsassessment,pa_commercial,pa_datasensitivity,pa_dataprotection,Origins/Selection of Dataset,Omitted Content and Reasons,Represented Groups,Image For Web,Keywords,pa_publishedonwiki,pa_publishedonflickr'

In [44]:
BL_SP_Fields = 'UniqueID,BL Dataset Provider,DAR,User Access Level,BL Labs Assistance ,Type Of Dataset,Year Span,Authors,Contributors,Creator,Additional Information,Language,Date Added,Last Time Updated,Contact Person,Department,Ownership,Location,Official URL,Related URLs,Location Notes,Is It Being Updated?,Any Issues With Access?,Image Res,Format,Files,Size,Divided Into Smaller Bits?,Quality,T&C Needed?,Rights Status,Rights Assessment,Commercial?,Data Sensitivity,Data Protection,Origins/Selection of Dataset,Omitted Content and Reasons,Represented Groups,Published on Wiki?,Published on Flickr?'
WP_Import_Fields = 'UniqueID,pa_bldatasetprovider,DAR,pa_useraccesslevel,pa_bllabsassistance,pa_typeofdataset,pa_yearspan,Authors,Contributors,Institution,Additional Information,pa_language,pa_date,Last Time Updated,Contact Person,pa_department,Ownership,Location,Official URL,Related URLs,Location Notes,pa_isitbeingupdated,Any Issues With Access,pa_imageres,pa_format,Files,Files Size,Divided Into Smaller Bits?,Quality,pa_tcneeded,pa_rightsdstatus,pa_rightsassessment,pa_commercial,pa_datasensitivity,pa_dataprotection,Origins/Selection of Dataset,Omitted Content and Reasons,Represented Groups,pa_publishedonwiki,pa_publishedonflickr'
Taxonomy = '0,1,0,1,1,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,1,1'

BL_SP_Fields_Array = BL_SP_Fields.split(",")
WP_Import_Fields_Array = WP_Import_Fields.split(",")
Taxonomy_Array = Taxonomy.split(",")

len(WP_Import_Fields_Array)

40

In [45]:
def atributes(rec):

    Atrib = '{'

    natrib = 0 #number of attributes

    i = 0
    for field in WP_Import_Fields_Array:
        if str(df.iloc[rec][BL_SP_Fields_Array[i]]).replace("nan", "") != '':
            Atrib += 's:' + str(len(field)) + ':"' + field.replace(" ", "-").lower() + '";a:6:{s:4:"name";s:' + str(len(field)) + ':"' + field + '";s:5:"value";s:' + str(len(str(df.iloc[rec][BL_SP_Fields_Array[i]]).replace("nan", ""))) + ':"' + str(df.iloc[rec][BL_SP_Fields_Array[i]]).replace("nan", "") + '";s:8:"position";i:'+ str(natrib) +';s:10:"is_visible";i:1;s:12:"is_variation";i:0;s:11:"is_taxonomy";i:' + Taxonomy_Array[i] + ';}'
            natrib += 1
        i += 1
    
    return Atrib,natrib

In [46]:
atributes(80)

('{s:8:"uniqueid";a:6:{s:4:"name";s:8:"UniqueID";s:5:"value";s:36:"f866aefa-b025-4675-b37d-44647649ba71";s:8:"position";i:0;s:10:"is_visible";i:1;s:12:"is_variation";i:0;s:11:"is_taxonomy";i:0;}s:20:"pa_bldatasetprovider";a:6:{s:4:"name";s:20:"pa_bldatasetprovider";s:5:"value";s:19:"Research Repository";s:8:"position";i:1;s:10:"is_visible";i:1;s:12:"is_variation";i:0;s:11:"is_taxonomy";i:1;}s:18:"pa_useraccesslevel";a:6:{s:4:"name";s:18:"pa_useraccesslevel";s:5:"value";s:15:"Online Download";s:8:"position";i:2;s:10:"is_visible";i:1;s:12:"is_variation";i:0;s:11:"is_taxonomy";i:1;}s:19:"pa_bllabsassistance";a:6:{s:4:"name";s:19:"pa_bllabsassistance";s:5:"value";s:10:"Not Needed";s:8:"position";i:3;s:10:"is_visible";i:1;s:12:"is_variation";i:0;s:11:"is_taxonomy";i:1;}s:11:"institution";a:6:{s:4:"name";s:11:"Institution";s:5:"value";s:15:"British Library";s:8:"position";i:4;s:10:"is_visible";i:1;s:12:"is_variation";i:0;s:11:"is_taxonomy";i:0;}s:11:"pa_language";a:6:{s:4:"name";s:11:"pa_lan

In [47]:
# we need this for the PubDate of "posts" to import: <pubDate>Mon, 20 Jun 2022 15:48:09 +0000</pubDate>

import datetime
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%a, %d %b %Y %H:%M:%S +0000")) # see https://strftime.org/

Current date and time : 
Fri, 29 Jul 2022 23:20:10 +0000


#### Everything ready to generate the XML for the import file: 

In [48]:
#df = df.reset_index()  # make sure indexes pair with number of rows

XML = '<?xml version="1.0" encoding="UTF-8"?>\n<rss version="2.0" xmlns:excerpt="http://wordpress.org/export/1.2/excerpt/" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:wfw="http://wellformedweb.org/CommentAPI/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:wp="http://wordpress.org/export/1.2/">\n  <channel>\n<title>BL Labs</title>\n<link>http://labs.biblios.tech</link>\n<description>Experiment with British Library\'s Digital Collections and Data</description>\n<pubDate>' + now.strftime("%a, %d %b %Y %H:%M:%S +0000") + '</pubDate>\n<language>en-GB</language>\n<wp:wxr_version>1.2</wp:wxr_version>\n<wp:base_site_url>http://labs.biblios.tech</wp:base_site_url>\n<wp:base_blog_url>http://labs.biblios.tech</wp:base_blog_url>\n'

nrec = 0

for rec, row in df.iterrows():
    print(nrec, row['Dataset Title'])
    
    # Dataset Title
    XML += '<item>\n  <title><![CDATA['+ str(df.iloc[rec]["Dataset Title"]) +']]></title>\n  <dc:creator>bladmin</dc:creator>\n'

    # Short Description
    XML += '  <excerpt:encoded><![CDATA['+ str(df.iloc[rec]["Short Description"]) +']]></excerpt:encoded>\n'

    XML += '  <wp:post_type><![CDATA[product]]></wp:post_type>\n  <wp:status><![CDATA[publish]]></wp:status>\n  <wp:is_sticky>0</wp:is_sticky>\n'
    XML += '  <category domain="product_cat" nicename="datasets"><![CDATA[Datasets]]></category>\n'
    XML += '  <category domain="product_type" nicename="external"><![CDATA[external]]></category>\n'

    # keywords / tags
    kws = str(df.iloc[rec]["Keywords"]).split(", ")
    for kw in kws:
        XML += '  <category domain="product_tag" nicename="'+ kw.replace(" ", "") + '"><![CDATA['+ kw +']]></category>\n'


    ## Products Atributes used to FILTER results:

    # Date Added
    #XML += '  <category domain="pa_date" nicename="' + df.iloc[rec]["Date Added"] + '"><![CDATA[' + df.iloc[rec]["Date Added"] + ']]></category>\n'
    # BL Dataset Provider
    #XML += '  <category domain="pa_bldatasetprovider" nicename="' + df.iloc[rec]["BL Dataset Provider"].replace(" ", "").lower() + '"><![CDATA[' + df.iloc[rec]["BL Dataset Provider"] + ']]></category>\n'  
    #Rights Assessment    
    #XML += '  <category domain="pa_rightsassessment" nicename="' + df.iloc[rec]["Rights Assessment"].replace(" ", "").lower() + '"><![CDATA[' + df.iloc[rec]["Rights Assessment"] + ']]></category>\n'    
    #Location    
    #XML += '  <category domain="pa_location" nicename="' + df.iloc[rec]["Location"].replace(" ", "").lower() + '"><![CDATA[' + df.iloc[rec]["Location"] + ']]></category>\n'    
    #Availability Status
    #XML += '  <category domain="pa_availabilitystatus" nicename="' + df.iloc[rec]["Availability Status"].replace(" ", "").lower() + '"><![CDATA[' + df.iloc[rec]["Availability Status"] + ']]></category>\n' 

    i = 0
    for Taxonomy in Taxonomy_Array:
        if Taxonomy == '1':
            XML += ' <category domain="'+ WP_Import_Fields_Array[i] +'" nicename="' + str(df.iloc[rec][BL_SP_Fields_Array[i]]).replace(" ", "").replace("nan", "").lower() + '"><![CDATA[' + str(df.iloc[rec][BL_SP_Fields_Array[i]]).replace("nan", "") + ']]></category>\n'
        i += 1


    ## Products Atributes that won't be used to FILTER results:

    #Authors
    XML += '  <wp:postmeta>\n    <wp:meta_key>Authors</wp:meta_key>\n    <wp:meta_value><![CDATA[' + str(df.iloc[rec]["Authors"]) + ']]>\n  </wp:meta_value>\n  </wp:postmeta>\n'
    #Contributors
    XML += '  <wp:postmeta>\n    <wp:meta_key>Contributors</wp:meta_key>\n    <wp:meta_value><![CDATA[' + str(df.iloc[rec]["Contributors"]) + ']]>\n  </wp:meta_value>\n  </wp:postmeta>\n'
    #Institution
    XML += '  <wp:postmeta>\n    <wp:meta_key>Institution</wp:meta_key>\n    <wp:meta_value><![CDATA[' + str(df.iloc[rec]["Creator"]) + ']]>\n  </wp:meta_value>\n  </wp:postmeta>\n'
    #Language
    XML += '  <wp:postmeta>\n    <wp:meta_key>Language</wp:meta_key>\n    <wp:meta_value><![CDATA[' + str(df.iloc[rec]["Language"]) + ']]>\n  </wp:meta_value>\n  </wp:postmeta>\n'
    #Additional Information
    XML += '  <wp:postmeta>\n    <wp:meta_key>Additional Information</wp:meta_key>\n    <wp:meta_value><![CDATA[' + str(df.iloc[rec]["Additional Information"]) + ']]>\n  </wp:meta_value>\n  </wp:postmeta>\n'
    #Files
    XML += '  <wp:postmeta>\n    <wp:meta_key>Files</wp:meta_key>\n    <wp:meta_value><![CDATA[' + str(df.iloc[rec]["Files"]) + ']]>\n  </wp:meta_value>\n  </wp:postmeta>\n'
    #Files Size
    XML += '  <wp:postmeta>\n    <wp:meta_key>Files Size</wp:meta_key>\n    <wp:meta_value><![CDATA[' + str(df.iloc[rec]["Size"]) + ']]>\n  </wp:meta_value>\n  </wp:postmeta>\n'


    #Attributes /encoded

    XML += '  <wp:postmeta>\n	<wp:meta_key>_product_attributes</wp:meta_key>\n	<wp:meta_value><![CDATA[a:' + str(atributes(rec)[1]) + ':' + atributes(rec)[0] + '}]]></wp:meta_value>\n</wp:postmeta>\n'

    #link to record source (Official URL)
    XML += '  <wp:postmeta>\n    <wp:meta_key>_product_url</wp:meta_key>\n    <wp:meta_value><![CDATA[' + str(df.iloc[rec]["Official URL"]) + ']]></wp:meta_value>\n  </wp:postmeta>\n'

    #Button Text ("download")
    XML += '  <wp:postmeta>\n    <wp:meta_key>_button_text</wp:meta_key>\n    <wp:meta_value><![CDATA[Download]]></wp:meta_value>\n  </wp:postmeta>\n'

    XML += '</item>\n'
    nrec += 1
    
XML += '  </channel>\n</rss>'


0 AAS Card Catalogues: Turkish
1 Digitised Quarterly Lists PDFs and Metadata
2 Theatrical playbills from Britain and Ireland (OCR text only)
3 Early German Bookbindings in the British Library
4 Resources used in attribution of early German Bookbindings in the British Library
5 Siddham: the South Asia Inscriptions Database
6 Early Buddhist Rock-Cut Monasteries Of The Western Ghats: Locations
7 Early Buddhist Rock-Cut Monasteries Of The Western Ghats: Caves And Sculptures
8 Mapping Jewish Communities Of The Byzantine Empire
9 Mapping the Jewish Communities of the Byzantine Empire
10 Early Music Online
11 Results From A 2015 Survey On Git/Distributed Version Control At Imperial College London
12 epidoc-headers
13 LD bounding_box
14 Software Citation workshop panel session voting data
15 John Jaffray dataset; a hand list of printed books and scrap books compiled by Jaffray relating to bookbinding and trade unionism in (mainly) 19c Victorian London
16 Faber Music and Music Sales Publication

#### We don't even need to save the XML to a file:

1) Using WinSCP open the import.xml file from the server hosting the WordPress container (e.g., /home/ubuntu/dockers/wplabs/import.xml);

2) Copy the XML bellow and paste into that file:


In [50]:
print(XML)

<?xml version="1.0" encoding="UTF-8"?>
<rss version="2.0" xmlns:excerpt="http://wordpress.org/export/1.2/excerpt/" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:wfw="http://wellformedweb.org/CommentAPI/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:wp="http://wordpress.org/export/1.2/">
  <channel>
<title>BL Labs</title>
<link>http://labs.biblios.tech</link>
<description>Experiment with British Library's Digital Collections and Data</description>
<pubDate>Fri, 29 Jul 2022 23:20:10 +0000</pubDate>
<language>en-GB</language>
<wp:wxr_version>1.2</wp:wxr_version>
<wp:base_site_url>http://labs.biblios.tech</wp:base_site_url>
<wp:base_blog_url>http://labs.biblios.tech</wp:base_blog_url>
<item>
  <title><![CDATA[AAS Card Catalogues: Turkish]]></title>
  <dc:creator>bladmin</dc:creator>
  <excerpt:encoded><![CDATA[This dataset contains digitised microfilms of Turkish card catalogues.]]></excerpt:encoded>
  <wp:post_type><![CDATA[product]]></wp:post_type>
  <wp:status><![C

3) Save the file (it will automatically upload / update it at the server);

4) Copy the file from the host file system to the container, via a SSH session to the server:

    sudo docker cp /home/ubuntu/dockers/wplabs/import.xml wplabs:/var/www/html/
    sudo docker exec wplabs bash -c 'cd /var/www/html/ && wp import import.xml --allow-root --authors=skip &'
    

#### BONUS: Map of the datasets IDs with the Public URL at the WordPress site

In [80]:
for rec, row in df.iterrows():
    url = row['Dataset Title'].lower().replace(" ", "-").replace("/", "-").replace("--", "-").replace("-–-", "-")
    print(row['UniqueID'] + ',"https://labs.biblios.tech/product/' + url+'"')


16f3f849-2456-4581-9f23-c9ee60103b1d,"https://labs.biblios.tech/product/aas-card-catalogues:-turkish"
8de070f6-bb21-4d3f-89de-7c3930934d2f,"https://labs.biblios.tech/product/digitised-quarterly-lists-pdfs-and-metadata"
807f0563-025e-46be-a251-1ad1ba477981,"https://labs.biblios.tech/product/theatrical-playbills-from-britain-and-ireland-(ocr-text-only)"
cdb004c6-4e5d-4df9-b648-b33ea33b35dd,"https://labs.biblios.tech/product/early-german-bookbindings-in-the-british-library"
da95a80b-a051-459f-9913-9d0c7424e6b0,"https://labs.biblios.tech/product/resources-used-in-attribution-of-early-german-bookbindings-in-the-british-library"
661ecabb-8e7b-4d83-b2d0-35918350090f,"https://labs.biblios.tech/product/siddham:-the-south-asia-inscriptions-database"
272b8709-6f16-4ab1-8398-6bdd72c33300,"https://labs.biblios.tech/product/early-buddhist-rock-cut-monasteries-of-the-western-ghats:-locations"
c9551e1f-85af-492b-90c9-c294bf77b50e,"https://labs.biblios.tech/product/early-buddhist-rock-cut-monasteries-o